In [1]:
import sys  
sys.path.insert(0, '/content/drive/MyDrive/overoptimism')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install optuna

     |████████████████████████████████| 296kB 9.1MB/s 
     |████████████████████████████████| 174kB 20.7MB/s 
     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 81kB 6.8MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 112kB 16.2MB/s 
     |████████████████████████████████| 143kB 15.4MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-cp37-none-any.whl size=11107 sha256=f18e9c2f6c6204bad73acdd9b000e5cfdec2588713a07c322c7ec14a6cce73c7
  Stored in directory: /root/.cache/pip/wheels/25/af/b8/3407109267803f4015e1ee2ff23be0c8c19ce4008665931ee1
Successfully built pyperclip


In [4]:
%load_ext autoreload
%autoreload 2

In [13]:
def getBlobDensities(n_centers):
  return np.round([3/n_centers * x for x in range(1, n_centers + 1)], decimals=2)

In [5]:
def objective(trial):
    seed = 0

    rock_results = []
    other_results = []

    n_samples = trial.suggest_int('n_samples', 1, 16) * 100
    jitter = trial.suggest_int('jitter', 1, 20) * 0.01
    n_features = trial.suggest_int('n_features', 2, 20)
    n_centers = trial.suggest_int('n_centers', 2, 10)

    datasets = []

    for s in range(seed, seed+10):
        datasets.append(make_blobs(n_samples=n_samples, centers=n_centers, n_features=n_features, cluster_std=getBlobDensities(n_centers), random_state=s))
        
    step = 0
    for dataset in datasets: 
        run = {}
        run['run_id'] = run_id
        run['step'] = step
        run['n_samples'] = n_samples
        run['n_centers'] = n_centers
        run['n_features'] = n_features
        run['jitter'] = jitter
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])
        kmeans = KMeans(n_clusters=n_centers, random_state=seed).fit(data).labels_
        run['k'] = n_centers
        eps, min_pts = (0.2, 4)
        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = min_pts
        spectral = SpectralClustering(n_clusters=n_centers).fit(data).labels_
        bandwidth = estimate_bandwidth(data)
        run['bandwidth'] = bandwidth
        mean_shift = MeanShift(bandwidth=bandwidth).fit(data).labels_

        rock = ROCK(tmax=15).fit(data).labels_

        gt = dataset[1]
        rock_results.append(AMI(rock, gt))
        other_results.append(np.max([AMI(kmeans, gt), AMI(dbscan, gt), AMI(spectral, gt), AMI(mean_shift, gt)]))

        run['ROCK'] = AMI(rock, gt)
        run['K_MEANS'] = AMI(kmeans, gt)
        run['DBSCAN'] = AMI(dbscan, gt)
        run['SPECTRAL'] = AMI(spectral, gt)
        run['MEAN_SHIFT'] = AMI(mean_shift, gt)

        step += 1

        experiment.append(run)

    return np.mean(rock_results) - np.mean(other_results)

In [6]:
experiment = []
run_id = 0

In [7]:
import optuna
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

from util import dbscan_init

import logging
import sys

In [10]:
study_name = 'den_blobs_2'

In [11]:
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True,  direction="maximize")

[I 2021-05-04 07:25:40,254] A new study created in RDB with name: den_blobs_2


In [14]:
study.optimize(objective, n_trials=100)

[I 2021-05-04 07:26:12,312] Trial 1 finished with value: -0.26417013175765325 and parameters: {'n_samples': 2, 'jitter': 2, 'n_features': 12, 'n_centers': 3}. Best is trial 1 with value: -0.26417013175765325.
[I 2021-05-04 07:27:05,863] Trial 2 finished with value: -0.6580580228406615 and parameters: {'n_samples': 8, 'jitter': 8, 'n_features': 20, 'n_centers': 8}. Best is trial 1 with value: -0.26417013175765325.
[I 2021-05-04 07:28:46,892] Trial 3 finished with value: -0.7386750846249961 and parameters: {'n_samples': 12, 'jitter': 19, 'n_features': 12, 'n_centers': 4}. Best is trial 1 with value: -0.26417013175765325.
[I 2021-05-04 07:29:29,241] Trial 4 finished with value: -0.523976049556726 and parameters: {'n_samples': 5, 'jitter': 13, 'n_features': 15, 'n_centers': 6}. Best is trial 1 with value: -0.26417013175765325.
[I 2021-05-04 07:29:48,998] Trial 5 finished with value: -0.040117229613198746 and parameters: {'n_samples': 1, 'jitter': 20, 'n_features': 3, 'n_centers': 3}. Best 

In [ ]:
pd.DataFrame(experiment).to_csv('den_blobs.csv')